In [37]:
## Datas ###
to_be_matched = [ "BATCH_REFERENCE", "TRANSACTION_REFERENCE","BUYER_NAME", "SUPPLIER_NAME" , "INVOICE_REFERENCE", "PO_REFERENCE",
                    "INVOICE_DATE", "INVOICE_AMOUNT", "INVOICE_CURRENCY", "MATURITY_DATE", "PAYMENT_DATE", "TENOR", "NET_AMOUNT",
                    "GROSS_AMOUNT", "DISCOUNT_AMOUNT", "ADJUSTMENT_AMOUNT", "ADJUSTMENT_REASON_CODE","PAYMENT_TERM", "STATUS", "NOTES"]

target = [ "batchReference", "transactionReference", "buyerName", 
            "supplierName", "invoiceReference", "poReference", 
            "invoiceDate", "invoiceAmount", "invoiceCurrency", "maturityDate", 
            "paymentDate", "tenor", "netAmount", "grossAmount", 
            "discountAmount", "adjustmentAmount", "adjustmentReasonCode", "paymentTerm", "status", "notes"]


In [38]:
import numpy as np
import pandas as pd
import re

In [39]:
dataset = pd.DataFrame({"to_be_matched":to_be_matched,"target":target})

In [52]:
dataset.head()

,to_be_matched,target
0,BATCH_REFERENCE,batchReference
1,TRANSACTION,transactionReference
2,BUYER_NAME,buyerName
3,SUPPLIER_NAME,supplierName
4,INVOICE_REFERENCE,invoiceReference


In [41]:
### Cleaning the to be matched column ###
def clean_strings(string):
    string = re.sub('[^a-zA-Z0-9.?]',' ',string)
    string = string.replace(' ','').lstrip().rstrip()
    return string.lower()

dataset["to_be_matched"].apply(clean_strings)

0           batchreference
1              transaction
2                buyername
3             suppliername
4         invoicereference
5              poreference
6              invoicedate
7            invoiceamount
8          invoicecurrency
9             maturitydate
10             paymentdate
11                   tenor
12               netamount
13             grossamount
14          discountamount
15        adjustmentamount
16    adjustmentreasoncode
17             paymentterm
18                  status
19                   notes
Name: to_be_matched, dtype: object

In [43]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )

    return (matrix[size_x - 1, size_y - 1])

def levenshtein_rate(product1, product2):
    distance = levenshtein(product1, product2)
    max_len = max(len(product1), len(product2))
    return 1 - (distance / max_len)

score_list = [levenshtein_rate(list(dataset["to_be_matched"].apply(clean_strings))[i],list(dataset["target"].apply(clean_strings))[i]) for i in range(0,len(dataset["to_be_matched"]))]
#score_list = [levenshtein_rate(list(dataset["to_be_matched"])[i],list(dataset["target"])[i]) for i in range(0,len(dataset["to_be_matched"]))]
#round(sum(score_list)/len(score_list),2)
score_list

[1.0,
 0.55,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [44]:
from fuzzywuzzy import fuzz

fuzz_socre = [fuzz.token_sort_ratio(list(dataset["to_be_matched"].apply(clean_strings))[i],list(dataset["target"].apply(clean_strings))[i]) for i in range(0,len(dataset["to_be_matched"]))] 
fuzz_socre

C:\Users\eroynab\Documents\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[100,
 71,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100]

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer

def ngrams(string, n=3):
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

def cosine_sim(text1, text2):
    vectorizer = TfidfVectorizer(analyzer=ngrams)
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

In [81]:
vect_score = [cosine_sim(list(dataset["to_be_matched"].apply(clean_strings))[i],list(dataset["target"].apply(clean_strings))[i]) for i in range(0,len(dataset["to_be_matched"]))]

In [82]:
vect_score

[1.0000000000000002,
 0.5797386715376658,
 0.9999999999999997,
 0.9999999999999999,
 1.0,
 1.0000000000000002,
 1.0000000000000002,
 1.0000000000000002,
 0.9999999999999998,
 0.9999999999999999,
 1.0000000000000002,
 1.0000000000000002,
 0.9999999999999997,
 1.0000000000000002,
 1.0,
 0.9999999999999999,
 1.0000000000000002,
 1.0000000000000002,
 1.0,
 1.0000000000000002]

In [86]:
import spacy
! python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

doc1 = nlp(u'batchreference')
doc2 = nlp(u'BATCH_REFERENCE')

#print doc1.similarity(doc2)

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047114 sha256=7e74e92de3863eb24073cdbd1c2b80292ec02f9b3bba29c1dd1d334649ca174e
  Stored in directory: C:\Users\eroynab\AppData\Local\Temp\pip-ephem-wheel-cache-fbnmr_9f\wheels\b7\0d\f0\7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [91]:
import en_core_web_sm
nlp = en_core_web_sm.load()

doc1 = nlp(u'batchreference')
doc2 = nlp(u'BATCH_REFERENCE')

print (doc1.similarity(doc2))

0.2979752358504139


C:\Users\eroynab\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  import sys


In [98]:
from gensim.models import Word2Vec
sentences = [list(dataset["to_be_matched"].apply(clean_strings)),list(dataset["target"].apply(clean_strings))]
model = Word2Vec(sentences, min_count=1)
model.similarity('batchreference', clean_strings('BATCH#'))

C:\Users\eroynab\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  after removing the cwd from sys.path.


KeyError: "word 'batch' not in vocabulary"